In [1]:
import cv2
import numpy as np
from assets.color_pallet import iron_palette

In [4]:


# Load the image
image_path = '/Users/Mahir/ChoY/thermo_note/assets/T640.jpg'
image = cv2.imread(image_path)

# Display the image to select the ROI
cv2.imshow('Select ROI', image)
roi = cv2.selectROI('Select ROI', image)
cv2.destroyAllWindows()

# Crop the image to the selected ROI
roi_cropped = image[int(roi[1]):int(roi[1]+roi[3]), int(roi[0]):int(roi[0]+roi[2])]


def color_to_temp(rgb, temp_min=82.9, temp_max=137):
    hex_color = f"#{rgb[0]:02x}{rgb[1]:02x}{rgb[2]:02x}"
    # print(hex_color)
    try:
        index = iron_palette.index(hex_color)
        # Map the index linearly to the temperature range
        temp = temp_min + (index / (len(iron_palette) - 1)) * (temp_max - temp_min)
        return temp
    except ValueError:
        # Color not found, you might want to handle this differently
        return None

# Calculate average temperature in the ROI
temperatures = []
for row in roi_cropped:
    for pixel in row:
        tem= color_to_temp(pixel)
        if tem is None:
            continue
        temperatures.append(color_to_temp(pixel))

average_temperature = np.mean(temperatures)
print(f"Average Temperature: {average_temperature}°F")

# Show the cropped ROI and temperature
cv2.imshow('Cropped ROI', roi_cropped)
cv2.waitKey(0)
cv2.destroyAllWindows()


Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!


/Users/Mahir/ChoY/thermo_note/.venv/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/Mahir/ChoY/thermo_note/.venv/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Average Temperature: nan°F


In [3]:
image_path = '/Users/Mahir/ChoY/thermo_note/assets/TTT.jpeg'

image = cv2.imread(image_path)

# Display the image to select the ROI
cv2.imshow('Select ROI', image)
roi = cv2.selectROI('Select ROI', image)
cv2.destroyAllWindows()

# Crop the image to the selected ROI
roi_cropped = image[int(roi[1]):int(roi[1]+roi[3]), int(roi[0]):int(roi[0]+roi[2])]

# Define the color palette (simplified example)

min_temp = 22.1
max_temp = 41.6

# Function to find nearest color in palette and calculate temperature
def color_distance(hex1, hex2):
    # Simple RGB distance
    rgb1 = [int(hex1[i:i+2], 16) for i in range(1, len(hex1), 2)]
    rgb2 = [int(hex2[i:i+2], 16) for i in range(1, len(hex2), 2)]
    return sum((c1 - c2) ** 2 for c1, c2 in zip(rgb1, rgb2)) ** 0.5
def color_to_temperature(color):
    # Convert color from BGR to hex
    hex_color = '#{0:02x}{1:02x}{2:02x}'.format(color[2], color[1], color[0])

    # Find the nearest color in the palette
    closest_color = min(iron_palette, key=lambda x: color_distance(x, hex_color))
    index = iron_palette.index(closest_color)

    # Interpolate temperature
    return min_temp + (max_temp - min_temp) * (index / len(iron_palette))

# Calculate average temperature in the ROI
temperatures = []
for row in roi_cropped:
    for pixel in row:
        temperatures.append(color_to_temperature(pixel))

average_temperature = np.mean(temperatures)
print(f"Average Temperature: {average_temperature}°F")
def temperature_to_color(average_temp):
    index = int((average_temp - min_temp) / (max_temp - min_temp) * (len(iron_palette) - 1))
    return iron_palette[max(0, min(index, len(iron_palette) - 1))]



average_color = temperature_to_color(average_temperature)
print(f"Average Color: {average_color}")

# Convert hex color to RGB tuple
def hex_to_bgr(hex_color):
    rgb = tuple(int(hex_color[i:i+2], 16) for i in range(1, len(hex_color), 2))
    return (rgb[2], rgb[1], rgb[0])  # Convert RGB to BGR for OpenCV

# Create an image of the average color
color_image = np.zeros((100, 100, 3), dtype=np.uint8)
color_image[:] = hex_to_bgr(average_color)

# Display the color image
cv2.imshow('Average Color', color_image)

# Show the cropped ROI and temperature
cv2.imshow('Cropped ROI', roi_cropped)
while True:
    key = cv2.waitKey(1) & 0xFF
    if key == ord('c'):
        break

cv2.destroyAllWindows()



Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Average Temperature: 34.639896073902996°F
Average Color: #f68200


In [17]:
def click_event(event, x, y, flags, params):
    display_image = image.copy()  # Copy of the image for displaying with marked area

    if event == cv2.EVENT_LBUTTONDOWN:
        # Check if the clicked point is within an edge (non-zero in the edge image)
        if edges[y, x] != 0:
            # Find the contour that the point is within
            for cnt in contours:
                if cv2.pointPolygonTest(cnt, (x, y), False) >= 0:
                    cv2.fillPoly(mask, [cnt], 0)  # Exclude the area in the mask
                    cv2.fillPoly(display_image, [cnt], (0, 0, 255))  # Fill the area with red in display image
                    cv2.imshow('Area Marked', display_image)
                    break


def show_after(image,edges):
    # Find contours from the edges, using RETR_EXTERNAL to get only the outermost contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    mask = np.zeros_like(edges)

    # Fill the contours to create a full mask of the cursor area
    cv2.fillPoly(mask, contours, 255)
    # Invert the mask to use for transparency
    mask_inv = cv2.bitwise_not(mask)

    # Convert the original image to BGRA (adds an alpha channel)
    bgra = cv2.cvtColor(image, cv2.COLOR_BGR2BGRA)

    # Use the inverted mask to set the alpha channel where cursor is removed
    bgra[:, :, 3] = mask_inv

    # # Optionally, create an image with the cursor region as transparent
    # # Transparent where the cursor was (set alpha to 0)
    # bgra[:, :, 3] = np.where(dilated_edges > 0, 0, bgra[:, :, 3])

    # Save or display the image
    cv2.imshow('Image with Transparent Cursor', bgra)
    cv2.imwrite('image_with_transparent_cursor.png', bgra)

image_path = '/Users/Mahir/ChoY/thermo_note/assets/edge_detect_example3.png'
image = cv2.imread(image_path)

# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply Gaussian blur to reduce noise and improve edge detection
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# Apply Canny edge detector
edges = cv2.Canny(blurred, 50, 150)


# Optionally, use dilation to enhance edges (helpful if cursor edges are not connected)
kernel = np.ones((3,3), np.uint8)
dilated_edges = cv2.dilate(edges, kernel, iterations=1)

# Use the edges to create a mask (invert if necessary)
# Here, the assumption is cursor edges form a closed loop
mask = cv2.bitwise_not(dilated_edges)  # Invert if the cursor is filled by the edges

# Apply the mask to exclude the cursor from the image
image_without_cursor = cv2.bitwise_and(image, image, mask=mask)

# Display the results
cv2.imshow('Original Image', image)
cv2.imshow('Edgess', edges)

contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Setup mouse callback function to get user click
cv2.namedWindow('Edges')
cv2.setMouseCallback('Edges', click_event, None)

# Display the edge image waiting for user to select a point
cv2.imshow('Edges', edges)

cv2.imshow('Image without Cursor', image_without_cursor)
cv2.waitKey(0)
cv2.destroyAllWindows()

# # Check if the ROI was selected and process it
# if len(roi) == 2:
#     roi_cropped = image[min(roi[0][1], roi[1][1]):max(roi[0][1], roi[1][1]),
#                         min(roi[0][0], roi[1][0]):max(roi[0][0], roi[1][0])]
#     cv2.imshow("Cropped ROI", roi_cropped)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

: 